In [1]:
import nltk
import pandas as pd
from textblob import TextBlob
import logging
from gensim import corpora, models, similarities
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import os
import codecs
from sklearn import feature_extraction
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.learning_curve import learning_curve
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score,  roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import export_graphviz
from sklearn import svm, grid_search
from sklearn import tree
from sklearn.preprocessing import FunctionTransformer
import sklearn
import mpld3
import lda
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# import sys
# reload(sys)  
# sys.setdefaultencoding('Cp1252')

In [3]:
bl=pd.read_csv('data/bl.csv', na_values='DK', encoding='latin-1')
fy15q3= pd.read_csv('data/fy15q3.csv', na_values='DK', encoding='latin-1')
fy16= pd.read_csv('data/fy16.csv', na_values='DK', encoding='latin-1')

In [4]:
len(fy16)

3068

### Data Cleaning

In [5]:
fy16['geo']=fy16['geo'].fillna('NaM')
fy15q3['Geo']=fy15q3['Geo'].fillna('NaM')
bl['Region : Region (Global Area)']=bl['Region : Region (Global Area)'].fillna('NaM')

In [6]:
fy16.loc[fy16['translation.to.english.for..ltr.comment'].isnull(),
       'comment'] = fy16['ltr.comment']

fy15q3.loc[fy15q3['translation_to_1_q_ca_ltrfollowup_reporting_comment'].isnull(),
       'comment'] = fy15q3['LTR Comment']


In [7]:
fy16_1=fy16[[ 'email', 'geo','segment', 'role2', 'ltr','acct.mgt',
      'license.contract', 'best.practice', 'implement', 'product.qual', 'functionality', 'ease.of.use', 
      'tech.support','fiscalyear', 'quarter', 'ltr.comment','ltr_comment_theme1', 'ltr_comment_theme2',
      'ltr_comment_theme3']]

In [8]:
fy15q3['responsedate'][1061:1064]='NaN'
fy15q3['responsedate']=pd.to_datetime(pd.Series(fy15q3['responsedate']), format="%m/%d/%y %H:%M")


/Users/shani16/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [9]:
fy15q3['responseyr'], fy15q3['rmth'] = fy15q3['responsedate'].dt.year, fy15q3['responsedate'].dt.month

In [10]:
fy15q3['fiscalyear'] = 'NaN'
fy15q3.loc[fy15q3['rmth'] > 4, 'fiscalyear'] = "FY15"
fy15q3['quarter']="Q3"

In [11]:
fy15q3_1=fy15q3[[ 'Contact Email', 'Geo','Segment', 'Respondent Role', 'LTR', 
                 'INS - Overall Account Management',
      'INS - Licensing and Contract Process', 'INS - Best Practices Advice', 'INS - Product Implementation Process',
                 'INS - Overall Quality of Products', 'INS - Overall Functionality of Products', 
                 'INS - Overall Ease of Use', 
      'INS - Ability to Get Technical Support','fiscalyear', 'quarter', 'comment','LTR_Comment_Theme1 ',
                 'LTR_Comment_Theme2',
      'LTR_Comment_Theme3']]

In [12]:
fy15q3_1.columns=['email', 'geo','segment', 'role2', 'ltr','acct.mgt',
      'license.contract', 'best.practice', 'implement', 'product.qual', 'functionality', 'ease.of.use', 
      'tech.support','fiscalyear', 'quarter', 'ltr.comment','ltr_comment_theme1', 'ltr_comment_theme2',
      'ltr_comment_theme3']

In [13]:
bl.loc[bl['What can CA Technologies do to improve your likelihood to recommend us? - Translated'].isnull(),
       'new'] = bl['What can CA Technologies do to improve your likelihood to recommend us?']

In [14]:
bl.loc[bl['What one thing can CA Technologies to improve your likelihod to recommend us? Translated'].isnull(),
       'new1'] = bl['What one thing can CA Technologies to improve your likelihod to recommend us?']

In [15]:
bl.loc[bl['What can CA Technologies do to improve your likelihood to recommend us? - Translated.1'].isnull(),
       'new2'] = bl['What can CA Technologies do to improve your likelihood to recommend us?.1']

In [16]:
bl['ltr.comment']=pd.concat([bl['new'].dropna(), bl['new1'].dropna(), bl['new2'].dropna() ]).reindex_like(bl)
bl['theme1']=pd.concat([bl['LTR_Promoters_Theme1'].dropna(), bl['LTR_Passives_Theme1'].dropna(), bl['LTR_Detractors_Theme1'].dropna() ]).reindex_like(bl)
bl['theme2']=pd.concat([bl['LTR_Promoters_Theme2'].dropna(), bl['LTR_Passives_Theme2'].dropna(), bl['LTR_Detractors_Theme2'].dropna() ]).reindex_like(bl)
bl['theme3']=pd.concat([bl['LTR_Promoters_Theme3'].dropna(), bl['LTR_Passives_Theme3 : LTR_Passives_Theme3'].dropna(), bl['LTR_Detractors_Theme3'].dropna() ]).reindex_like(bl)

In [17]:
# bl.columns.tolist()

In [18]:
bl_1=bl[[ 'ContactEmail : ContactEmail', 'Region : Region (Global Area)','AccountType : AccountType',
         'Role_New_WP : Role', 'How likely are you to recommend CA Technologies ', 
        u'Overall Account Management\xe6\xe6\xe6\xe6\xe6\xe6\xe6\xe6\xe6\xe6\xe6\xe6\xe6\xe6',
      'Licensing and contract process', 'Best-practices advice', 'Product implementation process',
                 'Overall quality of products', 'Overall functionality of products', 
                 'Overall ease of use of CA Technologies products', 'Ability to get technical support from CA Technologies',
         'Fiscal Year', 'Quarter', 
         'ltr.comment','theme1', 'theme2', 'theme3']]

In [19]:
bl_1.columns=['email', 'geo','segment', 'role2', 'ltr','acct.mgt',
      'license.contract', 'best.practice', 'implement', 'product.qual', 'functionality', 'ease.of.use', 
      'tech.support','fiscalyear', 'quarter', 'ltr.comment','ltr_comment_theme1', 'ltr_comment_theme2',
      'ltr_comment_theme3']

In [20]:
fy15q3_1['role2'] = fy15q3_1['role2'].map({'End User': 'END_USER', 'Influencer': 'DM/INF', 'Decision Maker': 'DM/INF', 'Recommender': 'END_USER'   })

/Users/shani16/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [21]:
bl_1['role2'] = bl_1['role2'].map({'END': 'END_USER', 'INF': 'DM/INF', 'DM': 'DM/INF', 'PRO': 'END_USER'   })

/Users/shani16/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [22]:
fy16_1['fiscalyear'].value_counts()

FY16    3068
Name: fiscalyear, dtype: int64

In [23]:
frames= [bl_1, fy15q3_1, fy16_1]
df=pd.concat(frames)

In [24]:
df = df[pd.notnull(df['ltr.comment'])]

In [25]:
df.columns

Index([u'email', u'geo', u'segment', u'role2', u'ltr', u'acct.mgt',
       u'license.contract', u'best.practice', u'implement', u'product.qual',
       u'functionality', u'ease.of.use', u'tech.support', u'fiscalyear',
       u'quarter', u'ltr.comment', u'ltr_comment_theme1',
       u'ltr_comment_theme2', u'ltr_comment_theme3'],
      dtype='object')

In [26]:
df = df[(df.geo == 'NaM') ]

In [27]:
df=df.fillna(df.mean())

In [28]:
df.head()

,email,geo,segment,role2,ltr,acct.mgt,license.contract,best.practice,implement,product.qual,functionality,ease.of.use,tech.support,fiscalyear,quarter,ltr.comment,ltr_comment_theme1,ltr_comment_theme2,ltr_comment_theme3
1,alain.z.cote@dgag.ca,NaM,Named,END_USER,0,9,6.772884,4,2,0,6,0.000000,0.000000,FY14,Q2,"Improve the quality, stability and integration...",Product_Quality,Product_Scope,NaN
2,matt.merchant@ge.com,NaM,Platinum,DM/INF,9,10,8.000000,9,10,9,9,7.313038,7.602971,FY14,Q2,ITSM space is best for CA. Still work to be do...,Product_Scope,NaN,NaN
3,chris.schwind@firstdata.com,NaM,Platinum,END_USER,9,10,6.772884,9,7,8,8,8.000000,8.000000,FY14,Q2,"Service Assurance, Mainframe tools, Scheduling...",Product_Scope,NaN,NaN
4,dominic.nguyen@ge.com,NaM,Platinum,END_USER,3,8,6.772884,7,3,2,4,2.000000,3.000000,FY14,Q2,Needs to get product more stable and less bugs,Product_Quality,NaN,NaN
5,mz1223@att.com,NaM,Platinum,END_USER,7,8,6.772884,10,10,9,10,8.000000,2.000000,FY14,Q2,We primarily use Introscope under the APM suit...,Contracts_Negotiations,NaN,NaN


In [29]:
df['license.contract'].value_counts()

6.772884     1768
8.000000      263
7.000000      197
9.000000      173
6.000000      132
10.000000     107
5.000000      103
4.000000       48
3.000000       42
2.000000       41
1.000000       26
0.000000       26
Name: license.contract, dtype: int64

In [30]:
df['ltr.comment']=df['ltr.comment'].str.replace(r'\?', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'\r', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'\#NAME', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'n/a', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'na', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'-', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'.', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'___', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'_', '')

In [31]:
df['fiscalyear'].value_counts()

FY16    1236
FY15     987
FY14     701
Name: fiscalyear, dtype: int64

In [32]:
df.columns.tolist()

['email',
 'geo',
 'segment',
 'role2',
 'ltr',
 'acct.mgt',
 'license.contract',
 'best.practice',
 'implement',
 'product.qual',
 'functionality',
 'ease.of.use',
 'tech.support',
 'fiscalyear',
 'quarter',
 'ltr.comment',
 'ltr_comment_theme1',
 'ltr_comment_theme2',
 'ltr_comment_theme3']

## Clustering

In [33]:
df.columns=[ 'email',
 'geo',
 'segment',
 'role2',
 'ltr',
 'acctmgt',
 'licensecontract',
 'bestpractice',
 'implement',
 'productqual',
 'functionality',
 'easeofuse',
 'techsupport',
 'fiscalyear',
 'quarter',
 'ltrcomment',
 'ltr_comment_theme1',
 'ltr_comment_theme2',
 'ltr_comment_theme3']

In [34]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords += ['.', ',', '(', ')', "'", '"', 'ca', 'customer', 'need', 'products', 'would', 'like', 'de', 'sure',
             'nothing', 'cost', 'us', 'use', 'get', 'make', 'needs', 'need', 'customer','customers',
              'would', 'get', 'like', '30', 'also', 'many', 'day'
             '30', '1000', '10', '06', '100', '1000', '1000s', '11', '12', '13','14', '15', '18', '19', '1sr', '200',
             '2005', '2009', '2011', '2012', '20126', '2014', '2015', 'vs', 'you', 'we', 'know', 'even', 'see', 'zuk', 
              'fen' , 'gartner', 'rules', 'cas', 'year', 'nothing', 'look', 'sure', 'solutions', 'better', 'sure' ,
              'line', 'one','clear', 'products', 'product', 'tool']

In [136]:
num_clusters = 50  # sp.unique(labels).shape[0]

import nltk.stem
english_stemmer = nltk.stem.SnowballStemmer('english')

from sklearn.feature_extraction.text import TfidfVectorizer


class StemmedTfidfVectorizer(TfidfVectorizer):

    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

vectorizer = StemmedTfidfVectorizer(min_df=.0005, ngram_range=(1, 5), max_features=20, smooth_idf=True,
                                    use_idf=True,
                                    stop_words=stopwords, decode_error='ignore'
                                    )
vectorized = vectorizer.fit_transform(df.ltrcomment)


In [137]:
feature= vectorized.toarray()
feature_df = pd.DataFrame(feature, columns=vectorizer.get_feature_names())
feature_df[:1]

,good,help,implement,improv,integr,issu,magement,new,price,provid,qualiti,respons,servic,softwar,support,team,technolog,time,tool,work
0,0,0,0,0.48072,0.602548,0,0,0,0,0,0.637059,0,0,0,0,0,0,0,0,0


In [138]:
word_frequency = feature_df.sum().reset_index()
word_frequency.columns=['word', 'freq']

word_frequency=word_frequency.sort(columns='freq', ascending= False)
word_frequency[:10]


/Users/shani16/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,word,freq
14,support,394.337256
12,servic,245.315477
3,improv,220.789404
19,work,170.733716
9,provid,154.122050
15,team,139.568217
5,issu,137.961223
16,technolog,137.056910
8,price,128.596646
6,magement,127.327974


## Classification- Logistic Regression

In [139]:
# df_new1=df.join(topics)

In [140]:
df_new1=df

In [141]:
fy=pd.get_dummies(df_new1['fiscalyear'])
geo=pd.get_dummies(df_new1['geo'])
role=pd.get_dummies(df_new1['role2'])
quarter=pd.get_dummies(df_new1['quarter'])
segment=pd.get_dummies(df_new1['segment'])
df1=pd.concat([fy, geo, role, quarter, segment, df_new1], axis=1)
df1=pd.concat([fy, role, quarter, segment, df_new1], axis=1)

In [142]:
df1.columns.tolist()

[u'FY14',
 u'FY15',
 u'FY16',
 'DM/INF',
 'END_USER',
 u'Q1',
 u'Q2',
 u'Q3',
 u'Q4',
 u'Existing_Enterprise',
 u'Growth',
 u'Named',
 u'Platinum',
 'email',
 'geo',
 'segment',
 'role2',
 'ltr',
 'acctmgt',
 'licensecontract',
 'bestpractice',
 'implement',
 'productqual',
 'functionality',
 'easeofuse',
 'techsupport',
 'fiscalyear',
 'quarter',
 'ltrcomment',
 'ltr_comment_theme1',
 'ltr_comment_theme2',
 'ltr_comment_theme3']

In [143]:
df1['ltr_comment_theme1'].value_counts()

Product_Quality                             372
Support                                     370
Account_Team_Relationship                   286
Product_Scope                               230
Account Mgt                                 214
Contracts_Negotiations                      160
Account Management and Agent Support        157
Contracts                                   111
Product_Usability                           109
Product Mentions                             99
Product Scope                                98
Implementations_Upgrades                     64
Product Quality                              52
Services                                     45
Brand                                        41
Implementations/Upgrades                     37
CA Services                                  35
Product Usability                            34
Documentation                                18
Issue Resolution                             12
Unable to recommend                     

In [144]:
rename_theme=({'Product Quality': 'Product_Quality', 'Product_Quality': 'Product_Quality', 
               'Account_Team_Relationship': 'Account_Team_Relationship',
               'Contracts': 'Contracts','Product Mentions': 'Product Mentions',
               'Contracts_Negotiations': 'Contracts', 'Product_Scope': 'Product Scope',
            'Account Mgt': 'Account_Team_Relationship','Account Management and Agent Support': 'Support',
           'Support': 'Support','Product Scope': 'Product Scope','Brand': 'Brand','Documentation':'Product_Usability',
           'Issue Resolution': "Support",'Partners': "Brand",'Unable to recommend': "Unable to recommend",
            'Account_Team':'Account_Team_Relationship', 'Implementations/Upgrades': 'Implementations/Upgrades',
               'Implementations_Upgrades': 'Implementations/Upgrades',
            'CA Services': 'Services','Product Usability': 'Product_Usability',
               'Product_Usability': 'Product_Usability',
               'Self_service/KnowledgeBase/Website Topic': 'Self_service/KnowledgeBase/Website'})

In [145]:
df1['theme1']=df1['ltr_comment_theme1'].map(rename_theme)
df1['theme2']=df1['ltr_comment_theme2'].map(rename_theme)
df1['theme3']=df1['ltr_comment_theme3'].map(rename_theme)

In [146]:
rename_theme=({'Product Quality': 'Product_Quality', 'Product_Quality': 'Product_Quality', 
               'Account_Team_Relationship': 'Account_Team_Relationship',
               'Contracts': 'Account_Team_Relationship','Product Mentions': 'Product_Quality',
               'Contracts_Negotiations': 'Account_Team_Relationship', 'Product_Scope': 'Product_Quality',
            'Account Mgt': 'Account_Team_Relationship','Account Management and Agent Support': 'Support',
           'Support': 'Support','Product Scope': 'Product_Quality','Brand': 'Account_Team_Relationship',
               'Documentation':'Product_Quality',
           'Issue Resolution': "Support",'Partners': "Account_Team_Relationship",
               'Unable to recommend': "NaN",
            'Account_Team':'Account_Team_Relationship', 'Implementations/Upgrades': 'Support',
               'Implementations_Upgrades': 'Support',
            'CA Services': 'Support','Product Usability': 'Product_Quality',
               'Product_Usability': 'Product_Quality',
               'Self_service/KnowledgeBase/Website Topic': 'Support'})

In [147]:
df1['theme1']=df1['ltr_comment_theme1'].map(rename_theme)
df1['theme2']=df1['ltr_comment_theme2'].map(rename_theme)
df1['theme3']=df1['ltr_comment_theme3'].map(rename_theme)

In [148]:
df1['theme1'].value_counts()

Product_Quality              1012
Account_Team_Relationship     816
Support                       677
NaN                             3
Name: theme1, dtype: int64

In [149]:
df1.columns.tolist()

[u'FY14',
 u'FY15',
 u'FY16',
 'DM/INF',
 'END_USER',
 u'Q1',
 u'Q2',
 u'Q3',
 u'Q4',
 u'Existing_Enterprise',
 u'Growth',
 u'Named',
 u'Platinum',
 'email',
 'geo',
 'segment',
 'role2',
 'ltr',
 'acctmgt',
 'licensecontract',
 'bestpractice',
 'implement',
 'productqual',
 'functionality',
 'easeofuse',
 'techsupport',
 'fiscalyear',
 'quarter',
 'ltrcomment',
 'ltr_comment_theme1',
 'ltr_comment_theme2',
 'ltr_comment_theme3',
 'theme1',
 'theme2',
 'theme3']

In [150]:
# from sklearn.cross_validation import train_test_split
# X=new_data.iloc[:,0:53]
# y=new_data.iloc[:,54]
# # create 80%-20% train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [151]:
# est=LogisticRegression()
# est.fit(X_train, y_train)
# pred = est.predict(X_test)
# accuracy_score(pred, y_test)

In [152]:
# scores = [accuracy_score, precision_score, recall_score]
# models = [LogisticRegression(), SVC(probability = True), GaussianNB(), DecisionTreeClassifier(max_depth = 4), 
#           RandomForestClassifier()]
# def get_metrics(features, outcome, test=0.2):
    
#     train_X, test_X, train_y, test_y = train_test_split(features, outcome, test_size = test, random_state=444)

    
#     # create empty lists
#     LogisticReg = []
#     SVMC = []
#     GaussNB = []
#     DecisionTree = []
#     RandomForest = []
#     kNN9 = []
#     ExtraTrees = []
    
#     # list of lists
#     lists = [LogisticReg, SVMC, GaussNB, DecisionTree, RandomForest]
    
#     # populate lists with scores of each scoring method
#     for i, model in enumerate(models):
#         for score in scores:
#             est = model
#             est.fit(train_X, train_y)
#             pred = est.predict(test_X)
#             lists[i].append(score(test_y, pred))
        
#     # create a dataframe which aggregates the lists
#     scores_df = pd.DataFrame(data = [LogisticReg, SVMC, GaussNB, DecisionTree, RandomForest])
#     scores_df.index = ["LogisticReg", "SVMC", "GaussNB", "DecisionTree", "RandomForest"]
#     scores_df.columns = ["Accuracy", "Precision", "Recall"]
#     return scores_df

In [153]:
# get_metrics(X,y, 0.2)

In [154]:
df1.columns

Index([               u'FY14',                u'FY15',                u'FY16',
                    u'DM/INF',            u'END_USER',                  u'Q1',
                        u'Q2',                  u'Q3',                  u'Q4',
       u'Existing_Enterprise',              u'Growth',               u'Named',
                  u'Platinum',               u'email',                 u'geo',
                   u'segment',               u'role2',                 u'ltr',
                   u'acctmgt',     u'licensecontract',        u'bestpractice',
                 u'implement',         u'productqual',       u'functionality',
                 u'easeofuse',         u'techsupport',          u'fiscalyear',
                   u'quarter',          u'ltrcomment',  u'ltr_comment_theme1',
        u'ltr_comment_theme2',  u'ltr_comment_theme3',              u'theme1',
                    u'theme2',              u'theme3'],
      dtype='object')

## Classification - Naives Bayes/ SVM

In [155]:
df4=df1[[
 'email',
 'ltrcomment',
  'END_USER',
    'Platinum',
    u'ltr',
   u'acctmgt',
  u'licensecontract',
  u'bestpractice',
  u'implement',
  u'productqual',
  u'functionality',
  u'easeofuse',
 u'techsupport', 
     'theme1',
 'theme2',
 'theme3',]]

In [156]:
names=['Support', 'Account_Team_Relationship', 'Product_Quality']
for i in names:
    for column in df4:
        df4[i]=df4[i] = np.where((df4.theme1 ==i) | (df4.theme2 == i) | (df4.theme3==i), 1, 0)

/Users/shani16/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [157]:
df4.columns

Index([                    u'email',                u'ltrcomment',
                        u'END_USER',                  u'Platinum',
                             u'ltr',                   u'acctmgt',
                 u'licensecontract',              u'bestpractice',
                       u'implement',               u'productqual',
                   u'functionality',                 u'easeofuse',
                     u'techsupport',                    u'theme1',
                          u'theme2',                    u'theme3',
                         u'Support', u'Account_Team_Relationship',
                 u'Product_Quality'],
      dtype='object')

In [158]:
df4 = df4.rename(columns={'email': 'contactemail', 'implement': 'implementations'})

In [159]:
df5=feature_df.join(df4)

In [160]:
df5=df5.drop(['contactemail', 'ltrcomment', 'Support','Product_Quality', 'Account_Team_Relationship'], axis=1)
# df5=df5.dropna()

In [161]:
df5 = df5.replace(np.nan,' ', regex=True)

In [162]:
df5['themes'] = df5.theme1.map(str) + "," + df5.theme2  + ',' + df5.theme3

In [163]:
df5['themes']=df5['themes'].replace(',,',',', regex=True)
df5['themes']=df5['themes'].replace(',, ','', regex=True)
# df5['themes']= df5['themes'].replace(', ','', regex=True)
# df5['themes']= df5['themes'].replace(' ,','', regex=True)
df5['themes']= df5['themes'].replace(' ','', regex=True)
df5['themes'] = df5['themes'].replace('NaN','', regex=True)
df5['themes']=df5['themes'].replace(', ','', regex=True)


In [164]:
df5=df5[(df5.themes != '')]
df5=df5[(df5.themes != ",")]

In [165]:
rename_theme=({'Product_Quality,' : 'Product_Quality',
'Account_Team_Relationship,' : 'Account_Team_Relationship'  ,                                          
'Support,': 'Support',                                                                      
'Product_Quality,Product_Quality,' : 'Product_Quality',
'Support,Support,Support' : 'Support' ,                                                         
'Account_Team_Relationship,Product_Quality,':'Account_Team_Relationship,Product_Quality',            
'Support,Product_Quality,' : 'Support,Product_Quality',                                                          
'Product_Quality,Support,' : 'Support,Product_Quality',                                                          
'Account_Team_Relationship,Account_Team_Relationship,' : 'Account_Team_Relationship' ,                            
'Account_Team_Relationship,Support,' : 'Account_Team_Relationship,Support' ,                                               
'Support,Support,' :'Support'   ,  
'Support,Account_Team_Relationship,' : 'Account_Team_Relationship,Support' ,
'Account_Team_Relationship,Product_Quality,Product_Quality' : 'Account_Team_Relationship,Product_Quality',                         
'Account_Team_Relationship,Support,Product_Quality' : 'Account_Team_Relationship,Support,Product_Quality' ,                            
'Account_Team_Relationship,Account_Team_Relationship,Product_Quality' : 'Account_Team_Relationship,Product_Quality',
'Support,Support,Product_Quality'  : 'Support,Product_Quality',
'Support,Product_Quality,Product_Quality' :'Support,Product_Quality',                                          
'Product_Quality,Product_Quality,Product_Quality' : 'Product_Quality',                                   
'Product_Quality,Account_Team_Relationship, ' : 'Product_Quality,Account_Team_Relationship',                                        
'Account_Team_Relationship,Account_Team_Relationship,Support' : 'Account_Team_Relationship,Support',                      
'Account_Team_Relationship,Account_Team_Relationship,Account_Team_Relationship' : 'Account_Team_Relationship',     
'Support,Account_Team_Relationship,Support' :  'Account_Team_Relationship,Support',                                       
'Support,Support,Account_Team_Relationship' : 'Account_Team_Relationship,Support',                                         
'Account_Team_Relationship,Support,Account_Team_Relationship' : 'Account_Team_Relationship,Support',                       
',Support,': 'Support',                                                                         
'Account_Team_Relationship,Support,Support' : 'Account_Team_Relationship,Support',                                         
'Support,Account_Team_Relationship,Account_Team_Relationship' : 'Account_Team_Relationship,Support',                        
'Support,Account_Team_Relationship,Product_Quality' : 'Account_Team_Relationship,Support,Product_Quality',
'Account_Team_Relationship,Product_Quality,Support' : 'Account_Team_Relationship,Support,Product_Quality',  
'Product_Quality,Product_Quality,Support' :   'Support,Product_Quality',                                      
'Product_Quality,Support': 'Support,Product_Quality',                                                            
'Account_Team_Relationship,Product_Quality,Account_Team_Relationship' : 'Account_Team_Relationship,Product_Quality',
'Product_Quality,Support,Account_Team_Relationship' : 'Account_Team_Relationship,Support,Product_Quality', 
'Support,Product_Quality,Support' :  'Support,Product_Quality',                                             
'Product_Quality,Account_Team_Relationship' :   'Account_Team_Relationship,Product_Quality',                                        
',Account_Team_Relationship,' : 'Account_Team_Relationship' ,                                                       
'Product_Quality,Support,Support' : 'Support,Product_Quality',                                                  
',Product_Quality,' : 'Product_Quality',                                                                 
',Account_Team_Relationship,Account_Team_Relationship': 'Account_Team_Relationship',                              
',Support,Account_Team_Relationship':  'Account_Team_Relationship,Support',                                                
'Account_Team_Relationship,Product_Quality' :  'Account_Team_Relationship,Product_Quality',                                         
'Product_Quality,Product_Quality,Account_Team_Relationship ' : 'Account_Team_Relationship,Product_Quality',
'Product_Quality,Support,Product_Quality' : 'Support,Product_Quality',                                       
',Product_Quality,Product_Quality': 'Product_Quality',                                                  
'Product_Quality,Account_Team_Relationship,Account_Team_Relationship':'Account_Team_Relationship,Product_Quality',                 
'Support,Support' : 'Support' ,                                                                   
'Product_Quality,Account_Team_Relationship,Support':'Account_Team_Relationship,Support,Product_Quality', 
               'Product_Quality,,': 'Product_Quality', 
             'Account_Team_Relationship,,': 'Account_Team_Relationship', 
              'Support,,': 'Support'})

In [166]:
df5['themesa']=df5['themes'].map(rename_theme)

In [167]:
df5=df5[pd.notnull(df5['themesa'])]

In [168]:
df5=df5[(df5.theme1 != ' ')]

In [169]:
names=['Support', 'Account_Team_Relationship',  'Product_Quality', ]

In [170]:
for i in names:
    for column in df5:
        df5[i]=df5[i] = np.where((df5.theme1 ==i) | (df5.theme2 == i) | (df5.theme3==i), 1, 0)

In [171]:
df5.head()

,good,help,implement,improv,integr,issu,magement,new,price,provid,...,easeofuse,techsupport,theme1,theme2,theme3,themes,themesa,Support,Account_Team_Relationship,Product_Quality
0,0,0,0,0.48072,0.602548,0,0.000000,0,0,0,...,7.31304,7.60297,Product_Quality,Product_Quality,,"Product_Quality,Product_Quality,",Product_Quality,0,0,1
1,0,0,0,0.00000,0.000000,0,0.754589,0,0,0,...,0,0,Product_Quality,Product_Quality,,"Product_Quality,Product_Quality,",Product_Quality,0,0,1
1,0,0,0,0.00000,0.000000,0,0.754589,0,0,0,...,7.31304,7.60297,Account_Team_Relationship,,,"Account_Team_Relationship,,",Account_Team_Relationship,0,1,0
2,0,0,0,0.00000,0.000000,0,0.000000,0,0,0,...,7.31304,7.60297,Product_Quality,,,"Product_Quality,,",Product_Quality,0,0,1
2,0,0,0,0.00000,0.000000,0,0.000000,0,0,0,...,7.31304,7.60297,Account_Team_Relationship,,,"Account_Team_Relationship,,",Account_Team_Relationship,0,1,0


In [172]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing
from sklearn.metrics import jaccard_similarity_score, hamming_loss, zero_one_loss, confusion_matrix, classification_report


from sklearn.cross_validation import train_test_split
X=df5.iloc[:,0:20]
y=np.array(df5.iloc[:,36:39])
# create 80%-20% train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=444)

print y_test

classifier = OneVsRestClassifier(LinearSVC(class_weight='balanced', random_state=444,  penalty='l2',
                                           loss='squared_hinge', 
                                           dual=True, tol=0.0001, C=1.0, multi_class='ovr', fit_intercept=True,
                                           intercept_scaling=1, 
                                           max_iter=1000))

classifier.fit(X_train, y_train)
predicted = classifier.predict(X_test)
print predicted
print 'jaccard similarity', jaccard_similarity_score(y_test, predicted)
print 'hamming_loss',hamming_loss(y_test, predicted)
print(classification_report(y_test, predicted))

[[0 1 0]
 [1 0 0]
 [0 1 1]
 ..., 
 [0 1 0]
 [0 1 0]
 [0 0 1]]
[[1 1 1]
 [1 1 0]
 [0 1 0]
 ..., 
 [0 0 1]
 [1 0 1]
 [1 1 0]]
jaccard similarity 0.347794649313
hamming_loss 0.504699927693
             precision    recall  f1-score   support

          0       0.37      0.67      0.48       160
          1       0.37      0.60      0.46       177
          2       0.49      0.38      0.43       207

avg / total       0.42      0.53      0.45       544



In [173]:
y_test[:10]

array([[0, 1, 0],
       [1, 0, 0],
       [0, 1, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [1, 1, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [174]:
predicted[:10]

array([[1, 1, 1],
       [1, 1, 0],
       [0, 1, 0],
       [1, 1, 0],
       [1, 0, 1],
       [1, 1, 0],
       [0, 1, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 1]])

In [175]:
from sklearn import cross_validation
predicted = cross_validation.cross_val_predict(classifier, X, y, cv=20)

In [176]:
print("Accuracy: %0.2f (+/- %0.2f)" % (predicted.mean(), predicted.std() * 2))

Accuracy: 0.53 (+/- 1.00)
